In [0]:
%run ./MasterNoteBooks/MNB_Sales_Emp_Dep

In [0]:
## Create a UDF to generate a unique employee code combining Department, Location, and ID.

emp = load_employee()
dep = load_department()
sal = load_salary()

def UniqueEmployeeID (Department, Location, EmployeeID):
    slice_d = str(Department)[:]
    slice_e = str(EmployeeID)[:]
    slice_L = str(Location)[:]
    return f"{slice_e}_{slice_d}_{slice_L}"

udf_val = udf(UniqueEmployeeID,StringType())

join = (emp
        .join(sal,"EmployeeID")
        .join(dep,"EmployeeId")
        .withColumn("Unique",udf_val("Department", "EmployeeId", "Location"))
        .select("EmployeeId","Department","Location","Unique")
        )

display(join)


In [0]:
## Create a UDF to compute salary after hypothetical deductions

def deduction(AnnualSalary,AnnualBonus,Tax):
    return AnnualSalary + AnnualBonus -Tax

udf_val = udf(deduction,DoubleType())

join = (emp
        .join(sal,"EmployeeId")
        .withColumn("NetSalary",(udf_val("AnnualSalary","AnnualBonus","Tax")))
        .withColumn("NetSalary",round(col("NetSalary"),2))
        .select("EmployeeId","NetSalary")
)

display(join)

In [0]:
%sql
select 1